# Suspicious eventlog clear or configuration using wevtutil
Detects clearing or configuration of eventlogs uwing wevtutil. Might be used by ransomwares during the attack (seen by NotPetya and others)

## Rule Content
```
- title: Suspicious eventlog clear or configuration using wevtutil
  id: cc36992a-4671-4f21-a91d-6c2b72a2edf5
  description: Detects clearing or configuration of eventlogs uwing wevtutil. Might
    be used by ransomwares during the attack (seen by NotPetya and others)
  author: Ecco
  date: 2019/09/26
  tags:
  - attack.execution
  - attack.t1070
  - car.2016-04-002
  level: high
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection_binary_1:
      Image: '*\wevtutil.exe'
    selection_binary_2:
      OriginalFileName: wevtutil.exe
    selection_clear_1:
      CommandLine: '* cl *'
    selection_clear_2:
      CommandLine: '* clear-log *'
    selection_disable_1:
      CommandLine: '* sl *'
    selection_disable_2:
      CommandLine: '* set-log *'
    condition: (1 of selection_binary_*) and (1 of selection_clear_* or 1 of selection_disable_*)
  falsepositives:
  - Admin activity
  - Scripts and administrative tools used in the monitored environment

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\wevtutil.exe OR file_name_original:"wevtutil.exe") AND (process_command_line.keyword:*\ cl\ * OR process_command_line.keyword:*\ clear\-log\ * OR process_command_line.keyword:*\ sl\ * OR process_command_line.keyword:*\ set\-log\ *))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()